In [1]:
import os 
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
sys.path.insert(0, os.path.join(spark_home, 'python'))
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 2.7.11 (default, Dec  6 2015 18:57:58)
SparkSession available as 'spark'.


In [2]:
DatasetRaw = sc.textFile('soc-data.txt').cache()

In [3]:
DatasetRaw = DatasetRaw.map(lambda line: line.split("\t")).filter(lambda line: len(line)==2).map(lambda line: (int(line[0].strip()),[int(x.strip()) for x in line[1].split(',') if x != ''])).sortByKey()

In [4]:
cartesian = DatasetRaw.cartesian(DatasetRaw)

In [5]:
def filterPairs(x):
     # don't take into account pairs of a same node and pairs of already friends
    if (x[0][0] != x[1][0]) and (not x[0][0] in x[1][1]) and (not x[1][0] in x[0][1]):
        shared = len(list(set(x[0][1]).intersection(set(x[1][1]))))
        return (x[0][0], [x[1][0], shared])

filteredDatasetRaw = cartesian.map(lambda x: filterPairs(x))
filteredDataset = filteredDatasetRaw.filter(lambda x: x != None)

In [6]:
groupedDataset = filteredDataset.groupByKey().mapValues(list)

In [7]:
def mapFinalDataset(elem):
    recommendations = []
    src = elem[0]
    dst_commons = elem[1]
    for pair in dst_commons:
        if pair[1] > 10: # 10 is the minimum number of friends in common
            recommendations.append(pair[0])
    return (strsrc, recommendations)


finalDataset = groupedDataset.map(mapFinalDataset)

In [8]:
#finalDataset.collect()

In [ ]:
sc.stop()